<h1>Bank Customer Churn Analysis</h1>

<h4>Importing the dataset</h4>

In [7]:
import pandas as pd

df = pd.read_csv('../data/Customer-Churn-Records.csv')

df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Card Type,Point Earned
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1,1,2,DIAMOND,464
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,1,3,DIAMOND,456
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,1,3,DIAMOND,377
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0,5,GOLD,350
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,0,5,GOLD,425


<h6>Checking for missing values & preparing dataset</h6>

In [8]:
df.isnull().sum()   # check for missing values

#df.describe()   # summary statistics

RowNumber             0
CustomerId            0
Surname               0
CreditScore           0
Geography             0
Gender                0
Age                   0
Tenure                0
Balance               0
NumOfProducts         0
HasCrCard             0
IsActiveMember        0
EstimatedSalary       0
Exited                0
Complain              0
Satisfaction Score    0
Card Type             0
Point Earned          0
dtype: int64

In [9]:
df = df.dropna()   # drop missing values

##or fill missing values with mean
#df = df.fillna(df.mean())

<h5>Revoming unnecessary columns</h5>

In [10]:
from sklearn.preprocessing import StandardScaler

# Load dataset (assuming df is already loaded)
df = df.drop(['RowNumber', 'CustomerId', 'Surname', 'Geography','Gender'], axis=1)  # Drop irrelevant columns

# Select numerical columns
num_cols = df.select_dtypes(include=['float64', 'int64']).columns   

# Initialize StandardScaler
scaler = StandardScaler()

# Fit and transform only numerical columns
df[num_cols] = scaler.fit_transform(df[num_cols])

# Display the first few rows after scaling
print(df.head())


   CreditScore  Gender       Age    Tenure   Balance  NumOfProducts  \
0    -0.326221  Female  0.293517 -1.041760 -1.225848      -0.911583   
1    -0.440036  Female  0.198164 -1.387538  0.117350      -0.911583   
2    -1.536794  Female  0.293517  1.032908  1.333053       2.527057   
3     0.501521  Female  0.007457 -1.387538 -1.225848       0.807737   
4     2.063884  Female  0.388871 -1.041760  0.785728      -0.911583   

   HasCrCard  IsActiveMember  EstimatedSalary    Exited  Complain  \
0   0.646092        0.970243         0.021886  1.976555  1.972908   
1  -1.547768        0.970243         0.216534 -0.505931  1.972908   
2   0.646092       -1.030670         0.240687  1.976555  1.972908   
3  -1.547768       -1.030670        -0.108918 -0.505931 -0.506866   
4   0.646092        0.970243        -0.365276 -0.505931 -0.506866   

   Satisfaction Score Card Type  Point Earned  
0           -0.721130   DIAMOND     -0.630839  
1           -0.009816   DIAMOND     -0.666251  
2           -0

<h4>Split the dataset</h4>

In [11]:
#split the dataset

from sklearn.model_selection import train_test_split

# Define Features (X) and Target (y)
X = df.drop('Exited', axis=1)   #drop the target column exited
y = df['Exited']        #target column

# Split the dataset into training and testing sets (80% training, 20% testing) 
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=0)   

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)


(8000, 13) (2000, 13) (8000,) (2000,)


<h4>Train Classification Using Different Models</h4>

In [12]:
#logistic regression
from sklearn.linear_model import LogisticRegression
#desicion tree
from sklearn.tree import DecisionTreeClassifier 
#random forest
from sklearn.ensemble import RandomForestClassifier
#support vector machine
from sklearn.svm import SVC

#Train models using model.fit(X_train, y_train)

# Initialize the models
log_reg = LogisticRegression()
dec_tree = DecisionTreeClassifier()
ran_for = RandomForestClassifier()
svm = SVC()

# Train the models
log_reg.fit(x_train, y_train)
dec_tree.fit(x_train, y_train)
ran_for.fit(x_train, y_train)
svm.fit(x_train, y_train)

# Predict the target on the training and testing sets
y_train_pred_log_reg = log_reg.predict(x_train)
y_test_pred_log_reg = log_reg.predict(x_test)

y_train_pred_dec_tree = dec_tree.predict(x_train)
y_test_pred_dec_tree = dec_tree.predict(x_test)

y_train_pred_ran_for = ran_for.predict(x_train)
y_test_pred_ran_for = ran_for.predict(x_test)

y_train_pred_svm = svm.predict(x_train)
y_test_pred_svm = svm.predict(x_test)

# Evaluate the models using accuracy_score
from sklearn.metrics import accuracy_score

# Calculate the accuracy of the models
accuracy_log_reg_train = accuracy_score(y_train, y_train_pred_log_reg)
accuracy_log_reg_test = accuracy_score(y_test, y_test_pred_log_reg)

accuracy_dec_tree_train = accuracy_score(y_train, y_train_pred_dec_tree)
accuracy_dec_tree_test = accuracy_score(y_test, y_test_pred_dec_tree)

accuracy_ran_for_train = accuracy_score(y_train, y_train_pred_ran_for)
accuracy_ran_for_test = accuracy_score(y_test, y_test_pred_ran_for)

accuracy_svm_train = accuracy_score(y_train, y_train_pred_svm)
accuracy_svm_test = accuracy_score(y_test, y_test_pred_svm)

# Print the accuracies
print('Logistic Regression')
print('Training accuracy: ', accuracy_log_reg_train)
print('Testing accuracy: ', accuracy_log_reg_test)
print('\n')

print('Decision Tree')
print('Training accuracy: ', accuracy_dec_tree_train)
print('Testing accuracy: ', accuracy_dec_tree_test)
print('\n')

print('Random Forest')
print('Training accuracy: ', accuracy_ran_for_train)
print('Testing accuracy: ', accuracy_ran_for_test)
print('\n')

print('Support Vector Machine')
print('Training accuracy: ', accuracy_svm_train)
print('Testing accuracy: ', accuracy_svm_test)
print('\n')



ValueError: could not convert string to float: 'Female'